In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/crimes-in-boston/crime.csv",encoding = "ISO-8859-1")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().transpose()

#### At First Glance we see that there is some missing data and the shooting column has less than 1% of data, So it would be btter droping that column entirely

In [ ]:
fig=plt.figure(figsize=(10,8))
sns.heatmap(df.isnull(),cmap="viridis",yticklabels=False,cbar=False)

In [ ]:
#inspecting shooting column before dropping

In [ ]:
df["SHOOTING"].unique()

In [ ]:
df[df["SHOOTING"]=="Y"]

#### Replacing NaN with "N" would be a better solution instead of Dropping the column


In [ ]:
def c(x):
    if x=="Y":
        return "Y"
    else:
        return "N"
        

df["SHOOTING"]=df["SHOOTING"].apply(lambda x:c(x))

In [ ]:
fig=plt.figure(figsize=(7,4))
sns.countplot(x="SHOOTING",data=df)

In [ ]:
df["SHOOTING"].value_counts()

### LETS START OUR EDA
#### We start with the Incident_number which is just a id to the crime so theres nothing much to investigate in that
#### Next is the offense code 

In [ ]:
df["OFFENSE_CODE"].unique()

In [ ]:
len(df["OFFENSE_CODE"].unique())

### There are a total of 221 possible crimes a person can commit that can be categorized and 1 category called other where remaining crimes go

In [ ]:
print(df["OFFENSE_CODE"].value_counts().head(10))

#### The following are the offense codes of the 10 most commited crimes

In [ ]:
print(df["OFFENSE_CODE"].value_counts().head(10).plot(kind="bar"))

In [ ]:
#Now lets see about OFFENSE_CODE_GROUP

In [ ]:
df["OFFENSE_CODE_GROUP"].nunique()

#### There are 67 categories those 222 offense codes can be categorized into

In [ ]:
fig=plt.figure(figsize=(20,8))
sns.countplot(x="OFFENSE_CODE_GROUP",data=df,)
plt.tight_layout()
plt.xticks(rotation=90)

In [ ]:
df["OFFENSE_CODE_GROUP"].value_counts().head(10)

#### The following are the offense code groups of the 10 most commited crimes

In [ ]:
df["OFFENSE_CODE_GROUP"].value_counts().head(10).plot(kind="bar",cmap="ocean")

In [ ]:
df["DISTRICT"].nunique()

In [ ]:
df["DISTRICT"].unique()

#### We will ignore the NaN for now as we are just exploring the data

In [ ]:
df["DISTRICT"].value_counts().head(5)

In [ ]:
fig=plt.figure(figsize=(9,4))
sns.countplot(x="DISTRICT",data=df,palette="spring")

In [ ]:
# now lets analyse how each distric fares 

In [ ]:
od=df["OFFENSE_CODE_GROUP"].value_counts().head(5).index

In [ ]:
fig=plt.figure(figsize=(20,7))

sns.countplot(x="DISTRICT",data=df,hue="OFFENSE_CODE_GROUP",hue_order=od)

### From this graph we can understand 
   * #### Motor vehicle Accident Response is the most commited crime in almost all the ditricts 
   * #### The Distrcit D4 sees the most Larceny so people in that district should be more careful
     


In [ ]:
od2=df["SHOOTING"].value_counts().index[1]
fig=plt.figure(figsize=(20,7))
cp=sns.countplot(x="SHOOTING",data=df[df["SHOOTING"]=="Y"],hue="DISTRICT")


In [ ]:
### District with high chances of shooting happening

In [ ]:
df["DISTRICT"].unique()
li=['D14', 'C11', 'D4', 'B3', 'B2', 'C6', 'A1', 'E5', 'A7', 'E13',
       'E18', 'A15']

In [ ]:
for i in li:
    a=int((len(df[(df["SHOOTING"]=="Y") & (df["DISTRICT"]== i)].index)/len(df[(df["SHOOTING"]=="Y")].index))*100)
    print("The chance of Shooting happening in the District {} is almost {} %".format(i,a))

In [ ]:
df.groupby("YEAR").count()["OFFENSE_CODE_GROUP"]

### WE CAN SEE A INCREASING TREND IN CRIMES FROM 2015-2017
#### We have to note that this dataset reports crimes till from June 14, 2015 to September 3, 2018.So lets see how many incidents occur in a day for every year

In [ ]:
a=len(df[df["YEAR"]==2015].index)/201
print("Number of crimes in the year 2015 per day on an average is {}".format(a))
b=len(df[df["YEAR"]==2016].index)/366
print("Number of crimes in the year 2016 per day on an average is {}".format(b))
c=len(df[df["YEAR"]==2017].index)/365
print("Number of crimes in the year 2017 per day on an average is {}".format(c))
d=len(df[df["YEAR"]==2018].index)/246
print("Number of crimes in the year 2018 per day on an average is {}".format(d))

In [ ]:
fig=plt.figure(figsize=(20,7))
sns.countplot(x="DISTRICT",data=df,hue="YEAR",palette="mako")

### From this graph we can understand 
   * #### The B2 distrcit leads in crime cases almost every year
   * #### Even though the whole of 2018 isnt taken into account the crime rate compared to last year is better in 2018 as we saw in the above cell

In [ ]:
df["OFFENSE_CODE_GROUP"].value_counts().tail(10)

### Note that a funny instance of house being burgled and nothing being taken is reported 2 times ;)

In [ ]:
df["MONTH"].value_counts()

In [ ]:
sns.countplot("MONTH",data=df)

#### The most crime cases seem to be recorded in the month of August
#### The least crime cases seem to be recorded in the month of February

### After visualization we can say that crimes are committed mostly in the summer.

In [ ]:
fig=plt.figure(figsize=(20,7))
sns.countplot("MONTH",data=df,hue="OFFENSE_CODE_GROUP",hue_order=od)

In [ ]:
fig=plt.figure(figsize=(20,7))
sns.countplot(x="HOUR",data=df,hue="DAY_OF_WEEK",palette="viridis")

### We can see that there are more no.of crimes happening from 12pm-12am rather than 12am-12pm
#### Around 3pm-7pm is the time when most of the crimes occur ,so people should be extra cautious around that time

In [ ]:
df["DAY_OF_WEEK"].value_counts()

In [ ]:
fig=plt.figure(figsize=(20,7))
sns.countplot(x="DAY_OF_WEEK",data=df,palette="summer")


### As expected Sunday witnessed the least amount of crimes as most of the people tend to stay home 
### While Friday was the day which encountered the most number of crimes in the week

In [ ]:
df["UCR_PART"].nunique()

In [ ]:
fig=plt.figure(figsize=(20,7))
sns.countplot(x="UCR_PART",data=df)

### We see that Part Three Crimes are more common

In [ ]:
df["STREET"].fillna("Unknown", inplace = True) 

In [ ]:
fig=plt.figure(figsize=(20,7))
sns.countplot(y='STREET',order=df.STREET.value_counts().head(10).index,data=df)

In [ ]:
### 